# Usage of the solver

In [13]:
# General setup
from numpy.random import randint, choice

explain some intro stuff here maybe

In [280]:
# idk if random generation is the right idea,
# maybe a more systematic approach would be better.
# just trying things out for now.
class Model():
    def __init__(self, worlds: list = None, agents: list = None,
                 relations=None, valuations=None):
        self.worlds = worlds
        self.agents = agents
        self.relations = relations
        self.valuations = valuations
        if self.valuations is None:
            self.create_valuations()
        if self.relations is None:
            self.create_relations()
        if self.agents is None:
            self.create_agents()
        if self.worlds is None:
            self.create_worlds()

    def create_worlds(self, max: int = 20):
        # generate a random number of worlds
        self.worlds = []
        for i in range(randint(1, max + 1)):
            self.worlds.append(f"w{i}")
        return self.worlds

    def create_agents(self, max: int = 26):
        # generate a random number of agents
        self.agents = []
        for i in range(randint(1, max + 1)):
            self.agents.append(chr(ord('A')+i))
        return self.agents

    def create_relations(self):
        # generate random relations
        if self.worlds is None:
            self.create_worlds()
        if self.agents is None:
            self.create_agents()
        # This should make S5 models, doesn't do that yet
        self.relations = {}
        for a in self.agents:
            self.relations[a] = {}
            for world in self.worlds:
                self.relations[a][world] = {w for w in self.worlds
                                            if w == world
                                            or randint(0, 10) < 5}
        # print(self.relations)
        return self.relations

    def create_valuations(self, max: int = 3):
        # generate random valuations
        if self.worlds is None:
            self.create_worlds()

        atoms = [chr(ord('p')+i) for i in range(randint(1, max + 1))]
        self.valuations = {}
        for atom in atoms:
            self.valuations[atom] = {w for w in self.worlds
                                     if randint(0, 10) < 5}
        # print(self.valuations)
        return self.valuations

In [618]:
def create_formula(model: Model, ldepth:int = 0, hdepth:int = 12) -> str:
    formula = ""

    for _ in range(randint(3,hdepth)):
        if randint(5) == 1:
            formula += '!'
        formula += "K" + choice(model.agents)
    if True: #randint(10) != 1:
        if randint(5) == 1:
            formula += '!'
        formula += choice(list(model.valuations.keys()))
    else: 
        formula += '(' + create_formula(model) + ")&(" + create_formula(model) + ')'

    return formula 

In [603]:
class Solver:
    def __init__(self, formula, model: Model = None, verbose: bool = False):
        """
        Initialize the solver with a starting formula and world

        :param formula: A formula that serves as the announcement to the riddle
        :param model: The Kripke model used in the riddle
        :param verbose: Whether or not the solver should print what it's working on.
        """
        self.start = formula
        self.level = 0
        self.f = formula
        self.model = model if model else Model()
        self.truths = {}
        self.talks = verbose

    def _eval_formula(self, formula: str, world: str) -> bool:
        """
        Evaluates the truth value of a formula in a specific world in the model. 
        It does this recursively, storing valuations in a dictionary to look up later.
        :param formula: The formula to evaluate
        :param world: The world in which to evaluate this formula
        """
        current = world + formula
        # Check if we already know the valuation of this formula in this world
        if (truth := self.truths.get(current, None)) is not None:
            return truth
        
        # What is the thing we need to evaluate?
        match atom := formula[0]:
            case atom if atom in self.model.valuations:
                # Atoms are true in this world if the world is in the set of valuations related to this atom
                self.truths[current] = world in self.model.valuations.get(atom, set())
            case 'K':
                # To evaluate a K-operator, we need to know which agent supposedly knows what
                agent, sub_formula = formula[1], formula[2:]
                # Then, the subformula must hold in all worlds this agent can reach from the current world.
                self.truths[current] = all(self._eval_formula(sub_formula, connected_world) for connected_world in self.model.relations[agent][world])
            case '!':
                # The not-operator simply negates the valuation of the rest of the line
                self.truths[current] = not(self._eval_formula(formula[1:], world))
            case '(':
                # Brackets can only appear on their own or in combination with an "&" (and)
                # So all subformulas within the brackets must hold
                this_bracket = [sub_formula for i, sub_formula in self._parse_brackets(formula) if i == 0]
                self.truths[current] = all(self._eval_formula(sub_formula, world) for sub_formula in this_bracket)
        # We stored our findings in the hashmap, so we can return the same value.
        return self.truths[current]

    def _parse_brackets(self, string: str):
        """
        Helper function
        :param string: String to parse the brackets of
        """
        stack = []
        for i, c in enumerate(string):
            if c == '(':
                stack.append(i)             # Keep track of where the brackets are
            elif c == ')' and stack:
                start = stack.pop()         # Remember where the last bracket started
                yield (len(stack), string[start + 1: i])    # Return the depth and content of each bracket

    def _satisfying_worlds(self):
        return [w for w in self.model.worlds if self._eval_formula(self.f, w)]

    def _strip_outer_K(self):
        if self.f[0] == 'K':
            self.f = self.f[2:]
        elif self.f[0:2] == '!K':
            self.f = self.f[3:]
        else:
            return False
        return True


    def find_riddle(self):
        """
        Main function
        """
        answers = []

        while True:
            # Find all worlds that satisfy the current state of the riddle
            ws = self._satisfying_worlds()

            if self.talks:
                print("\n")
                print("Formula:", self.f)
                print("Possible worlds:", ws)
            
            # What is the answer to this riddle?
            if not ws:
                # No satisfying worlds means the announcement was not consistent
                if self.talks:
                    print("→ inconsistent announcement")
                answers.append(None)
            elif len(ws) == 1:
                # There is a single satisfying world
                if self.talks:
                    print("→ unique world:", ws[0])
                answers.append(ws[0])
            else:
                # There are multiple satisfying worlds
                answer = []
                # Find what we know about all the atoms
                for atom in self.model.valuations:
                    truths = [w in self.model.valuations[atom] for w in ws]
                    if all(truths):
                        # We know this atom is true
                        answer.append(atom)
                    elif not(any(truths)):
                        # We know this atom is false
                        answer.append(f"!{atom}")
                    else:
                        # If neither, we can't say anything about the atom
                        answer.append(f"{atom}?")
                if self.talks:
                    print(f"My answer is → {', '.join(answer)}")
                # Keep track of the answers given for this order of ToM
                answers.insert(0, ", ".join(answer))

            # If there's no more K-operators to remove, stop the loop
            if not self._strip_outer_K():
                break
        # unique_answers = [a for a in answers if a is not None]
        # if (len(unique_answers) == len(set(unique_answers))
        #         and len(unique_answers) == len(answers)):
        if len(answers) != 0 and len(answers) == len(set(answers)):
            print("\nUnique different answer found for each K operator removed.")
            print("Formula:", self.start)
            print("worlds:", self.model.worlds)
            print("relations:", self.model.relations)
            print("valuations:", self.model.valuations)
            print("Answers:", [a for a in answers])
            return True
        else:
            if self.talks: print("\t\tNo unique different answer found for each K operator removed.")
            return False


To Do: 
- make sure model x formula combinations aren't repeated?
- generate better random search. this is a mess currently.

In [622]:
for i in range(90):
    model = Model()
    formula = create_formula(model)
    Solver(formula, model).find_riddle()



Unique different answer found for each K operator removed.
Formula: KGKB!KFq
worlds: ['w0', 'w1', 'w2', 'w3', 'w4', 'w5', 'w6', 'w7', 'w8', 'w9']
relations: {'A': {'w0': {'w0', 'w7', 'w6', 'w8', 'w3', 'w9', 'w2'}, 'w1': {'w1', 'w7', 'w5', 'w6'}, 'w2': {'w7', 'w6', 'w4', 'w3', 'w9', 'w2'}, 'w3': {'w8', 'w0', 'w6', 'w3'}, 'w4': {'w7', 'w4', 'w8', 'w5', 'w9'}, 'w5': {'w0', 'w7', 'w6', 'w1', 'w5', 'w2'}, 'w6': {'w0', 'w7', 'w6', 'w1', 'w3'}, 'w7': {'w7', 'w3'}, 'w8': {'w6', 'w1', 'w4', 'w8', 'w3', 'w5', 'w9', 'w2'}, 'w9': {'w0', 'w7', 'w6', 'w4', 'w8', 'w3', 'w5', 'w9', 'w2'}}, 'B': {'w0': {'w0', 'w7', 'w6', 'w1', 'w4', 'w8', 'w3', 'w5', 'w2'}, 'w1': {'w0', 'w6', 'w1', 'w4', 'w5'}, 'w2': {'w6', 'w4', 'w8', 'w3', 'w5', 'w9', 'w2'}, 'w3': {'w1', 'w8', 'w6', 'w3'}, 'w4': {'w0', 'w7', 'w6', 'w4', 'w3', 'w5', 'w9', 'w2'}, 'w5': {'w7', 'w4', 'w8', 'w3', 'w5', 'w9'}, 'w6': {'w8', 'w7', 'w5', 'w6'}, 'w7': {'w7', 'w6', 'w4', 'w3', 'w5', 'w9', 'w2'}, 'w8': {'w8', 'w5', 'w6'}, 'w9': {'w7', 'w6', 'w4

In [ ]:
model = Model()
# Infinite loop, would not recommend running
while not Solver(create_formula(model), model, False).find_riddle():
    print('.', end='') # just to show that it's doing something

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [1]:
# my sandbox, remove this when we're done
form = 'KB!KB(p)&(!q)'
rel = {'A': {'w0': {'w0', 'w1'}, 'w1': {'w0', 'w1'}}, 'B':{'w0': {'w0'}, 'w1': {'w1'}}}
val = {'p': {'w0', 'w1'}, 'q': {'w0'}}
agents = ['A', 'B']
wor = ['w0', 'w1']


dict = {}

keyone = "hello".join("hi")
keytwo = "hello".join("hi")

dict["w1(!p)&(!q)"] = False
dict["w1!p"] = True

print(dict["w1(!p)&(!q)"])


print(next(iter([1])))

False
1
